In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio
import chart_studio.plotly as py
import plotly.express as px

In [3]:
state_filepath='/Users/melodyqian/Documents/GitHub/FindMyNuclearWaste/CSVs/State_Level_Demographics.csv'
statedf=pd.read_csv(state_filepath)

In [6]:
pd.DataFrame.head(statedf, n=10)

,Unnamed: 0,state_name,occupied_percent,house_value_median,household_income_median,age_median,malepercent,femalepercent,hispanic_percent,nonhispanic_percent,white_percent,black_percent,asian_percent,pacificislander_percent,native_percent,state
0,0,Alabama,85.0,195100,62027,39.3,1128001,884210,1.6,27.5,3.1,0.5,0.2,1.6,94.6,1
1,1,Alaska,81.8,333300,89336,35.6,178458,97815,20.2,5.2,3.6,1.0,0.3,2.3,92.9,2
2,2,Arizona,89.0,358900,76872,38.8,1594780,1349934,6.0,6.4,26.8,0.7,0.3,3.3,69.0,4
3,3,Arkansas,86.0,175300,58773,38.4,705879,525644,2.4,16.4,6.2,0.3,0.1,2.2,91.2,5
4,4,California,92.4,695400,96334,37.6,8729012,5994486,2.8,7.2,32.3,0.6,0.3,6.7,60.2,6
5,5,Colorado,91.4,502200,92470,37.5,1240882,882557,2.9,5.7,15.8,0.7,0.2,5.5,77.8,8
6,6,Connecticut,92.5,343200,93760,41.2,733998,650772,1.2,13.6,1.8,8.2,0.4,7.4,82.2,9
7,7,Delaware,86.5,326800,82855,41.5,210396,201255,1.4,25.1,3.6,3.1,0.2,3.8,89.3,10
8,8,District of Columbia,90.3,724600,106287,34.9,125675,85501,1.6,46.7,2.1,0.8,0.4,8.2,88.4,11
9,9,Florida,84.8,325000,71711,42.6,4305366,4630733,1.2,17.8,3.3,5.5,7.1,10.8,73.3,12


In [7]:
fig = px.bar(statedf, x="state_name", y=["asian_percent"], title="Wide-Form Input")
fig.show()

In [8]:
 fig.write_html("samplestatedata.html")